In [1]:
import numpy as np
import copy

In [ ]:
w = [5, 4, 75, 74]

In [ ]:
[]

In [67]:
class KPBB:
    def __init__(self, cap, values, weights): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.solution=[]

    def initial_solution(self):
        capacit_cur=0
        for i in range(len(self.values)):
            if self.weights[i]+capacit_cur<=self.capacity:
                self.solution.append(i)
                capacit_cur+=self.weights[i]
            return self.solution

    def voisinage(self):
        teta=np.range(len(self.values))

        while teta in self.solution:
            teta=np.random.randint(len(self.values))
        

        capacit_cur=sum(self.weights[self.solution])

        while capacit_cur+self.weights[teta]>self.capacity:
            self.solution.pop(np.random.randint(len(self.solution)))

        self.solution.append(teta)

    def evaluation(self):
        return np.sum(np.array(self.values)[self.solution])
    
    def temperature(self):
        pass
    def solve(self):
        init=self.initial_solution()
        T0=10000

        while T0<0:
            pass

values = [40, 50, 100, 95, 30]
weights = [2, 3, 1, 5, 3]
capacite = 10
kpp = KPBB(capacite, values, weights)


kpp.initial_solution()

[0]